In [1]:
# REQUIRES PYTHON 3.12.0
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.optimizers import Adam
import cv2
import random
from collections import deque
import time
import os
from ale_py import ALEInterface, roms


env = gym.make("ALE/Frogger-v5", render_mode='rgb_array')


def reduceFrame(frame): # Make frame B&W and slim it down to reduce compute time
    gar=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    crap=gar[12:-30, 7:-7]
    resized = cv2.resize(crap, (84, 110), interpolation=cv2.INTER_AREA)
    return resized 


A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
